In [20]:
import numpy as np
import cv2
import os
import dlib
import shutil
from tqdm import tqdm
from shutil import copyfile
from collections import Counter

In [ ]:
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
datFile =  'C:/Users/jungh/final_project/landmarks/shape_predictor_68_face_landmarks.dat'

# Dlib의 얼굴 인식기 초기화
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(datFile)
faces = detector(gray)

In [223]:
img_path = 'C:/Users/jungh/final_project/img_path' 
cool_path = 'C:/Users/jungh/final_project/classified_img/cool'
warm_path = 'C:/Users/jungh/final_project/classified_img/warm'
file_list = os.listdir(img_path)

In [224]:
# 쿨톤과 웜톤 수 초기화
cool_count = 0
warm_count = 0

## 저장한사진 웜쿨 분류

In [ ]:
img_path = 'C:/Users/jungh/final_project/img_path' 
file_list = os.listdir(img_path)
for file in tqdm(file_list, desc="Processing images"):
    # 이미지 파일 경로
    image_path = os.path.join(img_path, file)
    
    # 이미지 로드
    image = cv2.imread(image_path)

    # 피부 톤 추출 및 배경 처리
    img_HSV = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    HSV_mask = cv2.inRange(img_HSV, (0, 15, 0), (17, 170, 255))
    HSV_mask = cv2.morphologyEx(HSV_mask, cv2.MORPH_OPEN, np.ones((3, 3), np.uint8))
    
    img_YCrCb = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)
    YCrCb_mask = cv2.inRange(img_YCrCb, (0, 135, 85), (255, 180, 135))
    YCrCb_mask = cv2.morphologyEx(YCrCb_mask, cv2.MORPH_OPEN, np.ones((3, 3), np.uint8))
    
    global_mask = cv2.bitwise_and(YCrCb_mask, HSV_mask)
    global_mask = cv2.medianBlur(global_mask, 3)
    global_mask = cv2.morphologyEx(global_mask, cv2.MORPH_OPEN, np.ones((4, 4), np.uint8))
    
    global_result = cv2.bitwise_not(global_mask)
    global_result_color = cv2.bitwise_not(global_result)
    
    output_image = cv2.bitwise_and(image, image, mask=global_result_color)
    output_image[global_result_color == 0] = (255, 255, 255)
    
    hsv_image = cv2.cvtColor(output_image, cv2.COLOR_BGR2HSV)
    
    hue = hsv_image[:, :, 0]
    
    lab_image = cv2.cvtColor(output_image, cv2.COLOR_BGR2LAB)
    L, a, b = lab_image[:, :, 0], lab_image[:, :, 1], lab_image[:, :, 2]
    
    if np.mean(a)*1.018 > np.mean(b):
        cool_count += 1
        copyfile(image_path, os.path.join(cool_path, file))
    else:
        warm_count += 1
        copyfile(image_path, os.path.join(warm_path, file))
    
# 결과 출력
print("쿨톤 수:", cool_count)
print("웜톤 수:", warm_count)

1.02 곱한 경우 쿨톤 수: 8491 웜톤 수: 4708

1.018 곱한 경우 쿨톤 수: 6647 웜톤 수: 6552

# 웜쿨 분류한 사진 봄여름가을겨울 분류

## 머리카락만 남기고 나머지 배경을 하얀색으로 만들어주는 코드

### hair의 lab의 L 과 skin의 lab의 L의 차이에 따라 각각의 계절로 분류

# warm에서 spring와 fall 분류

In [244]:
img_path = 'C:/Users/jungh/final_project/classified_img/warm' 
spring_path = 'C:/Users/jungh/final_project/classified_img/warm/spring'
fall_path = 'C:/Users/jungh/final_project/classified_img/warm/fall'

file_list = os.listdir(img_path)

In [ ]:
spring = 0
fall = 0

In [ ]:
for file in tqdm(file_list, desc="Processing images"):
  image_path = os.path.join(img_path, file)
    
    # 이미지 로드
  image = cv2.imread(image_path)
  
  img_HSV = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

  HSV_mask = cv2.inRange(img_HSV, (0, 15, 0), (17, 170, 255))
  HSV_mask = cv2.morphologyEx(HSV_mask, cv2.MORPH_OPEN, np.ones((3, 3), np.uint8))

  img_YCrCb = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)
  YCrCb_mask = cv2.inRange(img_YCrCb, (0, 135, 85), (255, 180, 135))
  YCrCb_mask = cv2.morphologyEx(YCrCb_mask, cv2.MORPH_OPEN, np.ones((3, 3), np.uint8))

  global_mask = cv2.bitwise_and(YCrCb_mask, HSV_mask)
  global_mask = cv2.medianBlur(global_mask, 3)
  global_mask = cv2.morphologyEx(global_mask, cv2.MORPH_OPEN, np.ones((4, 4), np.uint8))

  global_result = cv2.bitwise_not(global_mask)
  global_result_color = cv2.bitwise_not(global_result)

  skin_output = cv2.bitwise_and(image, image, mask=global_result_color)

  # 헤어
  spatial_radius = 10
  color_radius = 10
  max_pyramid_level = 1

  # Mean Shift 세그멘테이션을 수행합니다.
  segmented = cv2.pyrMeanShiftFiltering(image, spatial_radius, color_radius, maxLevel=max_pyramid_level)

  # 세그멘테이션된 이미지를 그레이스케일로 변환합니다.
  segmented_gray = cv2.cvtColor(segmented, cv2.COLOR_BGR2GRAY)

  # 이진화를 수행하여 머리카락 영역을 추출합니다.
  _, hair_mask = cv2.threshold(segmented_gray, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)

  # 목 주변의 영역을 더욱 정확하게 제거하기 위해 모폴로지 연산을 적용합니다.
  kernel = np.ones((5, 5), np.uint8)
  hair_mask = cv2.morphologyEx(hair_mask, cv2.MORPH_OPEN, kernel)

  # 머리카락 영역을 원본 이미지에 적용하여 머리카락만 남기고 나머지는 흰색으로 처리합니다.
  hair_output = cv2.bitwise_and(image, image, mask=hair_mask)
  
  skin_lab = cv2.cvtColor(skin_output, cv2.COLOR_BGR2LAB)
  skin_brightness = np.mean(skin_lab[:, :, 0]) 

  hair_lab = cv2.cvtColor(hair_output, cv2.COLOR_BGR2LAB)
  hair_brightness = np.mean(hair_lab[:, :, 0])

  contrast = skin_brightness - hair_brightness

  if contrast < 50:
    fall += 1
    copyfile(image_path, os.path.join(fall_path, file))

  else:
    spring += 1
    copyfile(image_path, os.path.join(spring_path, file))

    
print(spring)
print(fall)

# cool에서 summer와 winter 분류

In [254]:
img_path = 'C:/Users/jungh/final_project/classified_img/cool' 
summer_path = 'C:/Users/jungh/final_project/classified_img/cool/summer'
winter_path = 'C:/Users/jungh/final_project/classified_img/cool/winter'

file_list = os.listdir(img_path)

In [255]:
summer = 0
winter = 0

In [ ]:
for file in tqdm(file_list, desc="Processing images"):
  image_path = os.path.join(img_path, file)
    
    # 이미지 로드
  image = cv2.imread(image_path)
  
  img_HSV = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

  HSV_mask = cv2.inRange(img_HSV, (0, 15, 0), (17, 170, 255))
  HSV_mask = cv2.morphologyEx(HSV_mask, cv2.MORPH_OPEN, np.ones((3, 3), np.uint8))

  img_YCrCb = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)
  YCrCb_mask = cv2.inRange(img_YCrCb, (0, 135, 85), (255, 180, 135))
  YCrCb_mask = cv2.morphologyEx(YCrCb_mask, cv2.MORPH_OPEN, np.ones((3, 3), np.uint8))

  global_mask = cv2.bitwise_and(YCrCb_mask, HSV_mask)
  global_mask = cv2.medianBlur(global_mask, 3)
  global_mask = cv2.morphologyEx(global_mask, cv2.MORPH_OPEN, np.ones((4, 4), np.uint8))

  global_result = cv2.bitwise_not(global_mask)
  global_result_color = cv2.bitwise_not(global_result)

  skin_output = cv2.bitwise_and(image, image, mask=global_result_color)

  # 헤어
  spatial_radius = 10
  color_radius = 10
  max_pyramid_level = 1

  # Mean Shift 세그멘테이션을 수행합니다.
  segmented = cv2.pyrMeanShiftFiltering(image, spatial_radius, color_radius, maxLevel=max_pyramid_level)

  # 세그멘테이션된 이미지를 그레이스케일로 변환합니다.
  segmented_gray = cv2.cvtColor(segmented, cv2.COLOR_BGR2GRAY)

  # 이진화를 수행하여 머리카락 영역을 추출합니다.
  _, hair_mask = cv2.threshold(segmented_gray, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)

  # 목 주변의 영역을 더욱 정확하게 제거하기 위해 모폴로지 연산을 적용합니다.
  kernel = np.ones((5, 5), np.uint8)
  hair_mask = cv2.morphologyEx(hair_mask, cv2.MORPH_OPEN, kernel)

  # 머리카락 영역을 원본 이미지에 적용하여 머리카락만 남기고 나머지는 흰색으로 처리합니다.
  hair_output = cv2.bitwise_and(image, image, mask=hair_mask)
  
  skin_lab = cv2.cvtColor(skin_output, cv2.COLOR_BGR2LAB)
  skin_brightness = np.mean(skin_lab[:, :, 0]) 

  hair_lab = cv2.cvtColor(hair_output, cv2.COLOR_BGR2LAB)
  hair_brightness = np.mean(hair_lab[:, :, 0])

  contrast = skin_brightness - hair_brightness

  if contrast < 50:
    summer += 1
    copyfile(image_path, os.path.join(summer_path, file))

  else:
    winter += 1
    copyfile(image_path, os.path.join(winter_path, file))

    
print(summer)
print(winter)